In [85]:
import pandas as pd
excel_file_path = 'data_removed_anomalies.xlsx'
df = pd.read_excel(excel_file_path)
df.columns = df.columns.str.strip()

In [86]:
data_columns = [
    'Wetland Type - Provincial Class',
    'Wetland Type - Federal Class',
    'Water Regime Indicator',
    'Specific Vegetation Type',
    '% Vegetation Cover for Specific Vegetation Cover Types',
    '% High Woody Canopy Cover (>5m)',
    '% Moss Cover',
    'Phragmites present (Y/N)',
    'Soil Type',
    '% of Surface Water Present',
    'Depth of Saturation (cm)',
    'Average Depth of Living Moss (cm)',
    'Average Total Depth of Organics (moss, organics, muck) (cm)',
    'Average Organic Depth (cm)',
    'Hydrogeomorphic Class'
]

results_columns = [

    'SR',
    'SR_Function',
    'SR_Benefit',
    'SR Benefit Rating',
    'PR',
    'PR_Function',
    'PR_Benefit',
    'PR Benefit Rating',
    'NR',
    'NR_Function',
    'NR_Benefit',
    'NR Benefit Rating',
    'WS',
    'WS_Function',
    'WS_Benefit',
    'WS Benefit Score',
]

# Split DataFrame into two based on column names
data_df = df[data_columns]
results_df = df[results_columns]


In [87]:
import pandas as pd

# Load the data

# Function to transform categorical classes into numerical values
def transform_categorical(column):
    unique_values = column.unique()
    mapping = {value: index for index, value in enumerate(unique_values)}
    return column.map(mapping)

# Function to divide continuous values into 5 equal-interval classes
def divide_into_classes(column):

    column = column.apply(lambda x: 500 if x == 'Deeper than probe' else x)
    min_val = column.min()
    max_val = column.max()
    interval = (max_val - min_val) / 5
    return pd.cut(column, bins=5, labels=[0, 1, 2, 3, 4], right=True)
    
def handle_continuous(column):
    column = column.apply(lambda x: 500 if x == 'Deeper than probe' else x)
    column = column.apply(lambda x: 500 if x == 'Deeper than probe ' else x)

    return column


# Function to transform number values into ranges
def transform_number_range(column):
    # Calculate the min and max values
    min_val = column.min()
    max_val = column.max()
    
    # Define the bin edges
    bin_edges = [min_val, (min_val + max_val) / 2, max_val]
    
    # Define the labels for the bins
    labels = [0,1]
    
    # Bin the column values
    ranges = pd.cut(column, bins=bin_edges, labels=labels, right=False)
    return ranges

# Columns where continuous values need to be divided into 5 classes
continuous_columns = ['Average Depth of Living Moss (cm)','Average Total Depth of Organics (moss, organics, muck) (cm)', 'Average Organic Depth (cm)']

# Loop through each column and transform accordingly
for col in data_df.columns:
    if col in continuous_columns:  # If column contains continuous values
        data_df[col] = handle_continuous(data_df[col])
        #data_df[col] = divide_into_classes(data_df[col])
    elif data_df[col].dtype == 'object':  # If column contains string values
        data_df[col] = transform_categorical(data_df[col])
    elif data_df[col].dtype in ['int64', 'float64']:  # If column contains numerical values
        data_df[col] = transform_number_range(data_df[col])
data_df.to_excel('input.xlsx', index=False)

C:\Users\marcb\AppData\Local\Temp\ipykernel_47056\1374814822.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df[col] = transform_categorical(data_df[col])
C:\Users\marcb\AppData\Local\Temp\ipykernel_47056\1374814822.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df[col] = transform_categorical(data_df[col])
C:\Users\marcb\AppData\Local\Temp\ipykernel_47056\1374814822.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [88]:
# Iterate through columns of results_df
for col in results_df.columns:
    # Check if the column contains "Lower", "Moderate", or "Higher" values
    if any(value in ['Lower', 'Moderate', 'Higher'] for value in results_df[col]):
        # Replace the values accordingly
        results_df[col].replace({'Lower': 0, 'Moderate': 1, 'Higher': 2}, inplace=True)


C:\Users\marcb\AppData\Local\Temp\ipykernel_47056\3765382641.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df[col].replace({'Lower': 0, 'Moderate': 1, 'Higher': 2}, inplace=True)


In [89]:
results_df.to_excel('out.xlsx', index=False)

In [90]:
combined_df = pd.concat([data_df, results_df], axis=1)


In [92]:
combined_df.to_excel('combined.xlsx', index=False)

In [93]:
combined_df

,Wetland Type - Provincial Class,Wetland Type - Federal Class,Water Regime Indicator,Specific Vegetation Type,% Vegetation Cover for Specific Vegetation Cover Types,% High Woody Canopy Cover (>5m),% Moss Cover,Phragmites present (Y/N),Soil Type,% of Surface Water Present,...,PR_Benefit,PR Benefit Rating,NR,NR_Function,NR_Benefit,NR Benefit Rating,WS,WS_Function,WS_Benefit,WS Benefit Score
0,0,0,0,0,0,0,0,0,0,0,...,10.000000,2,10.000000,2,10.000000,2,8.420000,2,3.740000,1
1,1,1,1,1,0,1,1,0,1,1,...,0.000000,0,3.560000,1,0.000000,0,4.460000,1,0.000000,0
2,1,1,2,2,1,1,1,1,1,2,...,7.013615,2,1.870019,0,10.000000,2,0.013552,0,4.070119,1
3,0,0,0,0,0,2,2,0,0,3,...,0.000000,0,10.000000,2,0.000000,0,7.430000,2,0.000000,0
4,0,2,0,0,0,0,0,0,0,0,...,10.000000,2,10.000000,2,10.000000,2,9.010000,2,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,5,9,2,2,1,1,1,1,1,2,...,2.262433,1,1.144885,0,3.568376,1,-0.690006,0,10.000000,2
206,3,6,3,6,0,3,1,0,1,4,...,10.000000,2,10.000000,2,10.000000,2,8.130000,2,10.000000,2
207,1,1,1,1,2,1,1,0,1,1,...,6.470000,2,1.970000,0,4.620000,1,0.880000,0,0.000000,0
208,4,6,1,5,2,2,1,0,1,5,...,10.000000,2,2.930000,1,10.000000,2,1.810000,0,10.000000,2
